In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import argparse
import importlib
import data

from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

importlib.reload(data)

In [ ]:
#load img
#to load whole dataset will need to write a torch Dataset class

import data; importlib.reload(data)
batch_size=8

datamodule=data.IRAVENDataModule(batch_size=batch_size)
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

print(train_dataloader.dataset.__len__(),test_dataloader.dataset.__len__(),val_dataloader.dataset.__len__())

datamodule=data.IRAVENDataModule(batch_size=batch_size,split=(90,10,0))
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

print(train_dataloader.dataset.__len__(),test_dataloader.dataset.__len__(),val_dataloader.dataset.__len__())


In [ ]:
import collections

train_dataset_fnames=train_dataloader.dataset.filenames
test_dataset_fnames=test_dataloader.dataset.filenames


def get_type_strs(structure):
    types=structure[3]
    types_str=eval(str(types)[1:])
    return types_str

types_str_dataset=[]
for file in test_dataset_fnames:
    structure=np.load(file)['structure']
    types_str_dataset.append(get_type_strs(structure))
    






    frequency=collections.Counter(types_str_dataset)


In [ ]:
t1=torch.full((5,5),4.0,requires_grad=True)
t2=torch.full((5,5),2.0,requires_grad=False)
t3=t1*t2
print(t3.requires_grad)

## Testing SCL model-mask divergence

In [ ]:
#testing masked scl

import masking,data,utils,models.SCL_model
from data import IRAVENDataModule
importlib.reload(masking)
importlib.reload(data)
importlib.reload(utils)
importlib.reload(models.SCL_model)

from models.SCL_model import SCLTrainingWrapper,SCL

#setup
train=False
test=True
logging=True
model_ckpt='/Users/iyngkarrankumar/Documents/AI/AVR-functional-modularity/SCL_pretrain_80.ckpt'
task_path='datasets/light'
save_freq= 10000
batch_size=8
split=(85,15,0)
device=torch.device('cpu')

#dataset setup
if 1:
    #task dataset
    path=task_path
    data_module=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module.prepare_data()
    data_module.setup(root_dir=path)
    train_dataloader_task,test_dataloader_task=data_module.train_dataloader(),data_module.test_dataloader()
    x,y,*rest=next(iter(train_dataloader_task))

    #NOT task dataset
    path_='datasets/originals_masking'
    data_module_=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module_.prepare_data()
    data_module_.setup(root_dir=path_)
    test_dataloader_not_task=data_module_.test_dataloader()

#setup model
if 1:
    scl_kwargs={
        "image_size":160,                            # size of image
        "set_size": 9,                               # number of questions + 1 answer
        "conv_channels": [1, 16, 16, 32, 32, 32],    # convolutional channel progression, 1 for greyscale, 3 for rgb
        "conv_output_dim": 80,                       # model dimension, the output dimension of the vision net
        "attr_heads": 10,                            # number of attribute heads
        "attr_net_hidden_dims": [128],               # attribute scatter transform MLP hidden dimension(s)
        "rel_heads": 80,                             # number of relationship heads
        "rel_net_hidden_dims": [64, 32, 5] 
    }
    model=SCL(**scl_kwargs)

    #load
    state_dict=utils.get_SCL_state_dict(model_ckpt)
    model.load_state_dict(state_dict)
    model.eval()
    print(f'Model training: {model.training}')

#initialise masked model
if 1: 
    init_kwargs={
        'model':model,
        'train_dataloader':train_dataloader_task,
        'test_dataloader1':test_dataloader_task,
        'test_dataloader2':test_dataloader_not_task,
        'device':device,
        'savedir':'model_ckpts/FFN',
    }

    masked_scl=masking.MaskedSCLModel(init_kwargs)

#train
if 1:
    train_kwargs={
        'alpha':1e-6,
        'n_epochs':2 if test else 10,
        'n_batches':20 if test else 'full',
        'val_every_n_steps':5 if test else 10,
        'eval_every_n_steps':1e10,
        'n_val_batches':2 if test else 100,
        'n_eval_batches':2 if test else 100,
        'save_freq':save_freq,
        'logging':logging,

        }

    if train:
        masked_scl.train(**train_kwargs)


In [ ]:
#testing masked scl

import masking,data,utils,models.SCL_model
from data import IRAVENDataModule
importlib.reload(masking)
importlib.reload(data)
importlib.reload(utils)
importlib.reload(models.SCL_model)

from models.SCL_model import SCLTrainingWrapper,SCL

#setup
train=False
test=True
logging=True
model_ckpt='/Users/iyngkarrankumar/Documents/AI/AVR-functional-modularity/SCL_pretrain_80.ckpt'
task_path='datasets/light'
save_freq= 10000
batch_size=8
split=(85,15,0)
device=torch.device('cpu')

#dataset setup
if 1:
    #task dataset
    path=task_path
    data_module=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module.prepare_data()
    data_module.setup(root_dir=path)
    train_dataloader_task,test_dataloader_task=data_module.train_dataloader(),data_module.test_dataloader()
    x,y,*rest=next(iter(train_dataloader_task))

    #NOT task dataset
    path_='datasets/originals_masking'
    data_module_=IRAVENDataModule(batch_size=batch_size,split=split)
    data_module_.prepare_data()
    data_module_.setup(root_dir=path_)
    test_dataloader_not_task=data_module_.test_dataloader()

#setup model
if 1:
    scl_kwargs={
        "image_size":160,                            # size of image
        "set_size": 9,                               # number of questions + 1 answer
        "conv_channels": [1, 16, 16, 32, 32, 32],    # convolutional channel progression, 1 for greyscale, 3 for rgb
        "conv_output_dim": 80,                       # model dimension, the output dimension of the vision net
        "attr_heads": 10,                            # number of attribute heads
        "attr_net_hidden_dims": [128],               # attribute scatter transform MLP hidden dimension(s)
        "rel_heads": 80,                             # number of relationship heads
        "rel_net_hidden_dims": [64, 32, 5] 
    }
    model=SCL(**scl_kwargs)

    #load
    state_dict=utils.get_SCL_state_dict(model_ckpt)
    model.load_state_dict(state_dict)
    model.eval()
    print(f'Model training: {model.training}')

#initialise masked model
if 1: 
    init_kwargs={
        'model':model,
        'train_dataloader':train_dataloader_task,
        'test_dataloader1':test_dataloader_task,
        'test_dataloader2':test_dataloader_not_task,
        'device':device,
        'savedir':'model_ckpts/FFN',
    }

    masked_scl=masking.MaskedSCLModel(init_kwargs)

#train
if 1:
    train_kwargs={
        'alpha':1e-6,
        'n_epochs':2 if test else 10,
        'n_batches':20 if test else 'full',
        'val_every_n_steps':5 if test else 10,
        'eval_every_n_steps':1e10,
        'n_val_batches':2 if test else 100,
        'n_eval_batches':2 if test else 100,
        'save_freq':save_freq,
        'logging':logging,

        }

    if train:
        masked_scl.train(**train_kwargs)
